In [17]:
import tensorflow as tf
from tensorflow import keras
import sklearn
import numpy as np
import os
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [18]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [19]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



#### tokenizer사용시 index가 1부터 시작한다는점 주의 

In [20]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level = True)
tokenizer.fit_on_texts(shakespeare_text)

In [21]:
tokenizer.texts_to_sequences("First")

[[20], [6], [9], [8], [3]]

In [22]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [23]:
max_id = len(tokenizer.word_index)    #고유 글자 개수
dataset_size = tokenizer.document_count    #전체 글자 개수

In [24]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [25]:
train_size = dataset_size * 90//100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

### 0 ~ 100, 1 ~ 101...의 데이터를 생성한다

In [28]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift = 1, drop_remainder = True)

In [32]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [33]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [34]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [35]:
dataset = dataset.prefetch(1)

In [36]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


# Char-RNN 

In [16]:
# model = keras.models.Sequential([
#     keras.layers.GRU(128, return_sequences = True, input_shape=[None, max_id],
#                     dropout = 0.2),
#     keras.layers.GRU(128, return_sequences=True, dropout=0.2),
#     keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
# ])
# model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
# history = model.fit(dataset, epochs=20)

In [38]:
model = keras.models.load_model('shakespeare.h5')

In [18]:
# model.save("shakespeare.h5")

In [39]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [152]:
X_new = preprocess(["How are yo"])
# Y_pred = model.predict_classes(X_new)
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

'u'

In [159]:
model.predict(X_new)[0, -1:, :].shape

(1, 39)

In [133]:
tokenizer.sequences_to_texts(Y_pred + 1)

['e u   i   e   y o u']

###  tokenizer는 index를 빼지 않고 학습시켰고 model은 index를 0부터 학습시켰으므로 
### +1같은 쓸모없어보이는? 과정이 추가된다

In [142]:
tokenizer.sequences_to_texts(Y_pred + 1)

['e u   i   e   y o u']

In [143]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [174]:
next_char("How are yo", temperature=1)

'u'

In [181]:
next_char("How are yo", temperature=50)

'f'

In [145]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text+=next_char(text, temperature)
    return text

In [184]:
print(complete_text("I am", temperature=1))

I am a fortunes,
and yea, you mrys'd up virtue of bein


In [185]:
print(complete_text("I am", temperature=2))

I am cromc: i
wived for lit fimcertca? you lidried wha


# stateful RNN 

In [28]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [29]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2,
                     dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [30]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [31]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50,
                    callbacks=[ResetStatesCallback()])

Epoch 1/50
313/313 [==============================] - 8s 25ms/step - loss: 2.6254
Epoch 2/50
313/313 [==============================] - 3s 10ms/step - loss: 2.2435
Epoch 3/50
313/313 [==============================] - 3s 10ms/step - loss: 2.1118
Epoch 4/50
313/313 [==============================] - 3s 10ms/step - loss: 2.0382
Epoch 5/50
313/313 [==============================] - 3s 10ms/step - loss: 1.9853
Epoch 6/50
313/313 [==============================] - 3s 10ms/step - loss: 1.9503
Epoch 7/50
313/313 [==============================] - 3s 10ms/step - loss: 1.9221
Epoch 8/50
313/313 [==============================] - 3s 10ms/step - loss: 1.8987
Epoch 9/50
313/313 [==============================] - 3s 10ms/step - loss: 1.8814
Epoch 10/50
313/313 [==============================] - 3s 10ms/step - loss: 1.8681
Epoch 11/50
313/313 [==============================] - 3s 10ms/step - loss: 1.8545
Epoch 12/50
313/313 [==============================] - 3s 10ms/step - loss: 1.8436
Epoch 13/50
3

# 감성 분석

In [257]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

C:\Users\hjhhi\anaconda3\envs\python3.6.8\lib\site-packages\tensorflow_core\python\keras\datasets\imdb.py:129: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\hjhhi\anaconda3\envs\python3.6.8\lib\site-packages\tensorflow_core\python\keras\datasets\imdb.py:130: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [258]:
###구두점 제외 소문자로 변환해서 빈도 수를 이용하여 정수와 매칭하였다

In [259]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [260]:
word_index = keras.datasets.imdb.get_word_index()

In [261]:
### 아마 0,1,2에는 이미 특수한 문자와 매핑되어있어서 +3 해주는 느낌이다

id_to_word = {id_ + 3:word for word, id_ in word_index.items()}

In [262]:
id_to_word

{34704: 'fawn',
 52009: 'tsukino',
 52010: 'nunnery',
 16819: 'sonja',
 63954: 'vani',
 1411: 'woods',
 16118: 'spiders',
 2348: 'hanging',
 2292: 'woody',
 52011: 'trawling',
 52012: "hold's",
 11310: 'comically',
 40833: 'localized',
 30571: 'disobeying',
 52013: "'royale",
 40834: "harpo's",
 52014: 'canet',
 19316: 'aileen',
 52015: 'acurately',
 52016: "diplomat's",
 25245: 'rickman',
 6749: 'arranged',
 52017: 'rumbustious',
 52018: 'familiarness',
 52019: "spider'",
 68807: 'hahahah',
 52020: "wood'",
 40836: 'transvestism',
 34705: "hangin'",
 2341: 'bringing',
 40837: 'seamier',
 34706: 'wooded',
 52021: 'bravora',
 16820: 'grueling',
 1639: 'wooden',
 16821: 'wednesday',
 52022: "'prix",
 34707: 'altagracia',
 52023: 'circuitry',
 11588: 'crotch',
 57769: 'busybody',
 52024: "tart'n'tangy",
 14132: 'burgade',
 52026: 'thrace',
 11041: "tom's",
 52028: 'snuggles',
 29117: 'francesco',
 52030: 'complainers',
 52128: 'templarios',
 40838: '272',
 52031: '273',
 52133: 'zaniacs',

In [263]:
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
#####"어찌구".join 은 스트링 구분자 어찌구를 삽입해서 문자열 리스트를 하나로 묶어준다
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

In [264]:
for id_ in X_train[0][:10]:
    print(id_to_word[id_])

<sos>
this
film
was
just
brilliant
casting
location
scenery
story


In [265]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples

In [266]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0 = Negative

Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0 = Negative



In [269]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>",b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"<[a-zA-Z]",b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [270]:
preprocess(X_batch, y_batch)

(<tf.Tensor: shape=(2, 53), dtype=string, numpy=
 array([[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie.',
         b"Don't", b'be', b'lured', b'in', b'by', b'Christopher',
         b'Walken', b'or', b'Michael', b'Ironside.', b'Both', b'are',
         b'great', b'actors,', b'but', b'this', b'must', b'simply', b'be',
         b'their', b'worst', b'role', b'in', b'history.', b'Even',
         b'their', b'great', b'acting', b'could', b'not', b'redeem',
         b'this', b"movie's", b'ridiculous', b'storyline.', b'This',
         b'movie', b'is', b'an', b'early', b'nineties', b'US',
         b'propaganda', b'pi', b'<pad>', b'<pad>', b'<pad>'],
        [b'I', b'have', b'been', b'known', b'to', b'fall', b'asleep',
         b'during', b'films,', b'but', b'this', b'is', b'usually', b'due',
         b'to', b'a', b'combination', b'of', b'things', b'including,',
         b'really', b'tired,', b'being', b'warm', b'and', b'comfortable',
         b'on', b'the', b'sette', b'and', b'havi

In [271]:
from collections import Counter
vocabulary = Counter()
for X_batch, y_bath in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [272]:
vocabulary.most_common()[:3]

[(b'<pad>', 230521), (b'the', 60472), (b'a', 38249)]

In [249]:
vocab_size = 10000
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

In [250]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype = tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [251]:
table.lookup(tf.constant([b"This movie was faaaaaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   36,    17,    12, 10749]], dtype=int64)>

In [252]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [253]:
for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)

tf.Tensor(
[[   36    12    30 ...     0     0     0]
 [    8    22    70 ...     0     0     0]
 [ 6307  8013     1 ...     0     0     0]
 ...
 [   36    17   108 ...     0     0     0]
 [ 2105  6094  5067 ...     0     0     0]
 [10727 10165     8 ...     0     0     0]], shape=(32, 185), dtype=int64)
tf.Tensor([0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0], shape=(32,), dtype=int64)


In [254]:
embed_size = 128
model=keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size, input_shape=[None]),
    keras.layers.GRU(128,return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 25s 32ms/step - loss: 0.6936 - accuracy: 0.4980
Epoch 2/5
782/782 [==============================] - 22s 28ms/step - loss: 0.6882 - accuracy: 0.5140
Epoch 3/5
782/782 [==============================] - 22s 28ms/step - loss: 0.4549 - accuracy: 0.7664
Epoch 4/5
782/782 [==============================] - 22s 28ms/step - loss: 0.2114 - accuracy: 0.9166
Epoch 5/5
782/782 [==============================] - 22s 28ms/step - loss: 0.1170 - accuracy: 0.9582


In [215]:
K = keras.backend
embed_size = 128
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)
model = keras.models.Model(inputs=[inputs], outputs=[outputs])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 25s 31ms/step - loss: 0.5609 - accuracy: 0.7038
Epoch 2/5
782/782 [==============================] - 18s 22ms/step - loss: 0.3734 - accuracy: 0.8420
Epoch 3/5
782/782 [==============================] - 20s 26ms/step - loss: 0.1928 - accuracy: 0.9308
Epoch 4/5
782/782 [==============================] - 20s 25ms/step - loss: 0.1239 - accuracy: 0.9569
Epoch 5/5
782/782 [==============================] - 21s 27ms/step - loss: 0.1112 - accuracy: 0.9591


# 인코더 디코더 

In [220]:
import tensorflow_addons as tfa

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler,
                                                 output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, initial_state=encoder_state,
    sequence_length=sequence_lengths)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.models.Model(
    inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
    outputs=[Y_proba])

In [221]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [222]:
X = np.random.randint(100, size=10*1000).reshape(1000, 10)
Y = np.random.randint(100, size=15*1000).reshape(1000, 15)
X_decoder = np.c_[np.zeros((1000, 1)), Y[:, :-1]]
seq_lengths = np.full([1000], 15)

history = model.fit([X, X_decoder, seq_lengths], Y, epochs=2)

Train on 1000 samples
Epoch 1/2
1000/1000 [==============================] - 5s 5ms/sample - loss: 6.1650
Epoch 2/2
1000/1000 [==============================] - 2s 2ms/sample - loss: 4.6668


In [255]:
embed_size = 128
model=keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size, input_shape=[None]),
    keras.layers.GRU(128,return_sequences=True),
    keras.layers.Bidirectional(keras.layers.GRU(128, return_sequences=True)),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 33s 42ms/step - loss: 0.5060 - accuracy: 0.7544
Epoch 2/5
782/782 [==============================] - 28s 36ms/step - loss: 0.3026 - accuracy: 0.8801
Epoch 3/5
782/782 [==============================] - 29s 37ms/step - loss: 0.2038 - accuracy: 0.9284
Epoch 4/5
782/782 [==============================] - 29s 37ms/step - loss: 0.1437 - accuracy: 0.9495
Epoch 5/5
782/782 [==============================] - 28s 36ms/step - loss: 0.1022 - accuracy: 0.9646


In [256]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, None, 128)         1408000   
_________________________________________________________________
gru_20 (GRU)                 (None, None, 128)         99072     
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 256)         198144    
_________________________________________________________________
dense_9 (Dense)              (None, None, 1)           257       
Total params: 1,705,473
Trainable params: 1,705,473
Non-trainable params: 0
_________________________________________________________________


In [275]:
beam_width = 10
decoder = tfa.seq2seq.beam_search_decoder.BeamSearchDecoder(
cell=decoder_cell, beam_width = beam_width, output_layer = output_layer)
decoder_initial_state = tfa.seq2seq.beam_search_decoder.tile_batch(encoder_state, multiplier = beam_width)
outputs, _, _ = decoder(
embedding_decoder, start_tokens = start_tokens, end_token = end_token, initial_state = decoder_initial_state)

NameError: name 'embedding_decoder' is not defined